In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!pip install kaggle

In [ ]:
!kaggle datasets download -d ercanavsar/images-of-strawberry-leaves-for-tipburn-detection


Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'


In [ ]:
import os
print(os.getcwd())

/content


In [ ]:
!pip install kaggle

# Make sure kaggle.json is uploaded and in the right place
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download dataset to current directory (/content)
!kaggle datasets download -d ercanavsar/images-of-strawberry-leaves-for-tipburn-detection

# Extract it
import zipfile
import os

dataset_path = "/content/images-of-strawberry-leaves-for-tipburn-detection.zip"
extract_path = "/content/strawberry_leaf_dataset"

with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Dataset downloaded and extracted successfully!")

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'


FileNotFoundError: [Errno 2] No such file or directory: '/content/images-of-strawberry-leaves-for-tipburn-detection.zip'

In [ ]:
import json

# Replace "your_kaggle_username" with your actual Kaggle username
# Replace "your_kaggle_key" with the token you just got

kaggle_json = {
    "username": "YOUR_KAGGLE_USERNAME",
    "key": "KGAT_d21da844c4bcc4bbd26378876da2d8ab"  # copy your token here
}

# Save it as kaggle.json
with open("/content/kaggle.json", "w") as f:
    json.dump(kaggle_json, f)

print("✅ kaggle.json created successfully!")

In [ ]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install kaggle

In [ ]:
!kaggle datasets download -d ercanavsar/images-of-strawberry-leaves-for-tipburn-detection

In [ ]:
import zipfile
import os

dataset_path = "images-of-strawberry-leaves-for-tipburn-detection.zip"
extract_path = "strawberry_leaf_dataset"

with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted successfully!")


In [ ]:
import os

data_dir = "strawberry_leaf_dataset"  # Make sure this is the correct path

for root, dirs, files in os.walk(data_dir):
    print(f"📂 Directory: {root}")
    for file in files[:5]:  # Print first 5 files in each folder
        print(f"  📄 {file}")



In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

# Correct the base directory
data_dir = "strawberry_leaf_dataset/dataset - strawberry leaves"

# Check available subfolders (classes)
categories = os.listdir(data_dir)
print("Categories:", categories)  # Should print ["healthy", "Calciumdeficiency"]

# Select an image from the 'healthy' category
sample_image_path = os.path.join(data_dir, "healthy", os.listdir(os.path.join(data_dir, "healthy"))[0])

# Read and display the image
sample_image = cv2.imread(sample_image_path)
if sample_image is None:
    print("⚠️ Error: Could not read the image file!")
else:
    sample_image = cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB)
    plt.imshow(sample_image)
    plt.axis("off")
    plt.show()


In [ ]:
import numpy as np
import cv2
from skimage.feature import graycomatrix, graycoprops

def extract_features(image_path):
    # Read and resize image
    img = cv2.imread(image_path)
    img = cv2.resize(img, (128, 128))  # Resize for consistency

    # Convert to grayscale
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # --- 1. Color Histogram Features ---
    hist_r = cv2.calcHist([img], [0], None, [256], [0, 256]).flatten()
    hist_g = cv2.calcHist([img], [1], None, [256], [0, 256]).flatten()
    hist_b = cv2.calcHist([img], [2], None, [256], [0, 256]).flatten()

    # --- 2. Texture Features (GLCM) ---
    glcm = graycomatrix(img_gray, distances=[1], angles=[0], symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]

    # --- 3. Edge & Shape Features (Canny Edges) ---
    edges = cv2.Canny(img_gray, 100, 200)  # Detect edges
    edge_pixels = np.sum(edges) / (128 * 128)  # Edge density

    # Combine all extracted features into one vector
    features = np.hstack([hist_r, hist_g, hist_b, contrast, energy, homogeneity, correlation, edge_pixels])

    return features


In [ ]:
import pandas as pd
import os

# Define dataset path
data_dir = "strawberry_leaf_dataset/dataset - strawberry leaves"

# Define classes (Healthy = 0, Tipburn/Deficiency = 1)
labels = {"healthy": 0, "Calciumdeficiency": 1}

X = []  # Feature matrix
y = []  # Labels

# Loop through both classes
for category, label in labels.items():
    folder_path = os.path.join(data_dir, category)

    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)

        # Extract features
        features = extract_features(img_path)
        X.append(features)
        y.append(label)

# Convert to NumPy arrays
X = np.array(X)
y = np.array(y)

# Print dataset shape
print(f"Feature matrix shape: {X.shape}")  # Should be (num_samples, num_features)
print(f"Labels shape: {y.shape}")


In [ ]:
df = pd.DataFrame(X)
df['label'] = y  # Add label column

# Save to CSV
df.to_csv("strawberry_leaf_features.csv", index=False)
print("✅ Features saved successfully!")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [ ]:
# Load dataset
df = pd.read_csv("strawberry_leaf_features.csv")

# Separate features (X) and labels (y)
X = df.drop(columns=["label"]).values
y = df["label"].values

# Split into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalize features (important for XGBoost)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("✅ Data loaded and preprocessed successfully!")


In [ ]:
# Train Random Forest Model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Train XGBoost Model
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42, use_label_encoder=False, eval_metric="logloss")
xgb_model.fit(X_train, y_train)

print("✅ Models trained successfully!")


In [ ]:
# Get prediction probabilities from both models
rf_probs = rf_model.predict_proba(X_test)
xgb_probs = xgb_model.predict_proba(X_test)

# Hybrid Model: Weighted Averaging (50% RF + 50% XGBoost)
hybrid_probs = (0.5 * rf_probs) + (0.5 * xgb_probs)

# Convert probabilities to final predictions
hybrid_preds = np.argmax(hybrid_probs, axis=1)


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Compute Confusion Matrix
conf_matrix = confusion_matrix(y_test, hybrid_preds)

# Plot Confusion Matrix
plt.figure(figsize=(6,4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["Healthy", "Tipburn"], yticklabels=["Healthy", "Tipburn"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix - Hybrid Model")
plt.show()

# Print Classification Report
print("🔹 Classification Report:")
print(classification_report(y_test, hybrid_preds))


In [ ]:
# Accuracy Score
accuracy = accuracy_score(y_test, hybrid_preds)
print(f"✅ Model Accuracy: {accuracy:.4f}")

# F1-Score (More balanced than accuracy)
from sklearn.metrics import f1_score
f1 = f1_score(y_test, hybrid_preds)
print(f"✅ Model F1-Score: {f1:.4f}")


In [ ]:
# Compute Accuracy
accuracy = accuracy_score(y_test, hybrid_preds)
print(f"✅ Model Accuracy: {accuracy:.4f}")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [ ]:
# Load extracted features from CSV
df = pd.read_csv("strawberry_leaf_features.csv")

# Separate features (X) and labels (y)
X = df.drop(columns=["label"]).values
y = df["label"].values

# Split into Training (80%) and Testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalize the dataset (important for XGBoost)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("✅ Data Loaded & Preprocessed Successfully!")


In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
print("✅ Random Forest Trained Successfully!")


In [ ]:
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42, use_label_encoder=False, eval_metric="logloss")
xgb_model.fit(X_train, y_train)
print("✅ XGBoost Trained Successfully!")


In [ ]:
# Define the Hybrid Model using Voting Classifier
hybrid_model = VotingClassifier(estimators=[
    ('random_forest', rf_model),
    ('xgboost', xgb_model)
], voting='hard')  # Hard Voting (Majority Vote)

# Train the Hybrid Model
hybrid_model.fit(X_train, y_train)
print("✅ Hybrid Model (Voting Classifier) Trained Successfully!")


In [ ]:
# Make Predictions
y_pred = hybrid_model.predict(X_test)


In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ Hybrid Model Accuracy: {accuracy:.4f}")


In [ ]:
print("🔹 Classification Report:")
print(classification_report(y_test, y_pred))


In [ ]:
# Compute Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Plot Confusion Matrix
plt.figure(figsize=(6,4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["Healthy", "Tipburn"], yticklabels=["Healthy", "Tipburn"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix - Hybrid Model")
plt.show()


In [ ]:
from sklearn.model_selection import GridSearchCV

rf_params = {
    "n_estimators": [100, 200, 300],
    "max_depth": [10, 20, 30, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

rf_grid = GridSearchCV(RandomForestClassifier(random_state=42), rf_params, cv=3, scoring="accuracy", n_jobs=-1)
rf_grid.fit(X_train, y_train)

best_rf = rf_grid.best_estimator_
print(f"✅ Best RF Params: {rf_grid.best_params_}")


In [ ]:
xgb_params = {
    "n_estimators": [100, 200, 300],
    "learning_rate": [0.01, 0.1, 0.2],
    "max_depth": [3, 5, 7],
    "subsample": [0.8, 1.0]
}

xgb_grid = GridSearchCV(XGBClassifier(random_state=42, use_label_encoder=False, eval_metric="logloss"),
                        xgb_params, cv=3, scoring="accuracy", n_jobs=-1)
xgb_grid.fit(X_train, y_train)

best_xgb = xgb_grid.best_estimator_
print(f"✅ Best XGBoost Params: {xgb_grid.best_params_}")


In [ ]:
optimized_hybrid = VotingClassifier(estimators=[
    ('random_forest', best_rf),
    ('xgboost', best_xgb)
], voting='hard')

optimized_hybrid.fit(X_train, y_train)
y_pred_optimized = optimized_hybrid.predict(X_test)

accuracy_optimized = accuracy_score(y_test, y_pred_optimized)
print(f"🚀 Optimized Hybrid Model Accuracy: {accuracy_optimized:.4f}")


In [ ]:
soft_voting_model = VotingClassifier(estimators=[
    ('random_forest', best_rf),
    ('xgboost', best_xgb)
], voting='soft')  # Change to 'soft'

soft_voting_model.fit(X_train, y_train)
y_pred_soft = soft_voting_model.predict(X_test)

accuracy_soft = accuracy_score(y_test, y_pred_soft)
print(f"🚀 Soft Voting Hybrid Model Accuracy: {accuracy_soft:.4f}")


In [ ]:
importances = best_rf.feature_importances_
feature_names = df.drop(columns=["label"]).columns

# Select Top 10 Features
important_features = pd.DataFrame({"Feature": feature_names, "Importance": importances})
important_features = important_features.sort_values(by="Importance", ascending=False).head(10)

# Use only these top 10 features for training
X_train_important = X_train[:, important_features.index]
X_test_important = X_test[:, important_features.index]

# Retrain the Hybrid Model with only important features
optimized_hybrid.fit(X_train_important, y_train)
y_pred_reduced = optimized_hybrid.predict(X_test_important)

accuracy_reduced = accuracy_score(y_test, y_pred_reduced)
print(f"🚀 Optimized Hybrid Model (Feature Selection) Accuracy: {accuracy_reduced:.4f}")


In [ ]:
import cv2
import imgaug.augmenters as iaa
import os

augmenters = iaa.Sequential([
    iaa.Fliplr(0.5),  # Horizontal Flip
    iaa.Affine(rotate=(-20, 20)),  # Rotation
    iaa.Multiply((0.8, 1.2))  # Brightness
])

def augment_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    augmented_image = augmenters.augment_image(image)
    return augmented_image

# Apply augmentation on training images and retrain the model


In [ ]:
# Select the top 10 features from training
X_test_important = X_test[:, important_features.index]

# Now predict with the optimized model
y_pred_optimized = optimized_hybrid.predict(X_test_important)
accuracy_optimized = accuracy_score(y_test, y_pred_optimized)

print(f"🚀 Optimized Hybrid Model Accuracy: {accuracy_optimized:.4f}")



In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    n_estimators=300,  # More trees = better learning (try 300-500)
    max_depth=30,      # Increase depth (adjust based on performance)
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
)


In [ ]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier(
    n_estimators=500,      # More trees
    learning_rate=0.05,    # Lower learning rate
    max_depth=10,          # Deeper trees
    subsample=0.8,         # Prevents overfitting
    colsample_bytree=0.8,
    random_state=42
)


In [ ]:
from sklearn.ensemble import VotingClassifier

hybrid_model = VotingClassifier(
    estimators=[('rf', rf_model), ('xgb', xgb_model)],
    voting='soft',   # Use probability-based voting
    weights=[1, 2]   # Give more weight to XGBoost (try tuning this)
)


In [ ]:
import xgboost as xgb

# Convert datasets into DMatrix (XGBoost's optimized data format)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)








In [ ]:
params = {
    "objective": "binary:logistic",  # Since it's a classification problem
    "eval_metric": "logloss",        # Log loss metric
    "max_depth": 6,                  # Depth of trees
    "eta": 0.05,                      # Learning rate
    "subsample": 0.8,                 # Prevents overfitting
    "colsample_bytree": 0.8
}


In [ ]:
evals = [(dtest, "eval")]  # Validation dataset

xgb_model = xgb.train(
    params,               # Model parameters
    dtrain,               # Training data
    num_boost_round=500,  # Maximum boosting rounds
    evals=evals,          # Validation set
    early_stopping_rounds=50,  # Stops training if no improvement
    verbose_eval=True
)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_test)


In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

xgb_model = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
xgb_model.fit(X_train, y_train)

xgb_preds = xgb_model.predict(X_test)


In [ ]:
from sklearn.ensemble import VotingClassifier

hybrid_model = VotingClassifier(
    estimators=[("rf", rf_model), ("xgb", xgb_model)],
    voting="hard"  # Majority vote
)

hybrid_model.fit(X_train, y_train)
hybrid_preds = hybrid_model.predict(X_test)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

hybrid_accuracy = accuracy_score(y_test, hybrid_preds)
print(f"✅ Hybrid Model Accuracy: {hybrid_accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, hybrid_preds))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, hybrid_preds))


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# Load dataset
df = pd.read_csv("strawberry_leaf_features.csv")

# Separate features and labels
X = df.drop(columns=["label"]).values
y = df["label"].values

# Train a Random Forest model to get feature importance
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X, y)

# Get the top 20 important features
importances = rf.feature_importances_
indices = np.argsort(importances)[-20:]  # Select top 20
X_selected = X[:, indices]

# Update dataset with selected features
df_selected = pd.DataFrame(X_selected)
df_selected['label'] = y

# Save the new dataset
df_selected.to_csv("strawberry_leaf_selected_features.csv", index=False)

print("✅ Feature selection done! Dataset updated with important features.")


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define parameter grid
rf_params = {
    "n_estimators": [200, 300, 400],
    "max_depth": [10, 20, 30],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

# Perform Grid Search
rf_grid = GridSearchCV(RandomForestClassifier(random_state=42), rf_params, cv=3, scoring="accuracy", n_jobs=-1)
rf_grid.fit(X_selected, y)

# Get the best model
best_rf = rf_grid.best_estimator_
print(f"✅ Best RF Params: {rf_grid.best_params_}")


In [ ]:
!pip install optuna


In [ ]:
import optuna
import xgboost as xgb
from sklearn.model_selection import cross_val_score

def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500, step=50),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0)
    }
    model = xgb.XGBClassifier(**params, random_state=42)
    score = cross_val_score(model, X_selected, y, cv=3, scoring="accuracy").mean()
    return score

# Run optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

# Get best parameters
best_xgb_params = study.best_params
print(f"✅ Best XGBoost Params: {best_xgb_params}")


In [ ]:
import cv2
import imgaug.augmenters as iaa
import os

augmenters = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Affine(rotate=(-20, 20)),
    iaa.Multiply((0.8, 1.2))
])

def augment_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    augmented_image = augmenters.augment_image(image)
    return augmented_image

print("✅ Data augmentation applied!")


In [ ]:
from sklearn.ensemble import IsolationForest

# Train an Isolation Forest to detect outliers
iso_forest = IsolationForest(contamination=0.05, random_state=42)
outliers = iso_forest.fit_predict(X_selected)

# Keep only normal data points
X_clean = X_selected[outliers == 1]
y_clean = y[outliers == 1]

print(f"✅ Outliers removed! New dataset size: {X_clean.shape}")


In [ ]:
from sklearn.ensemble import VotingClassifier

# Define the hybrid model with optimized parameters
optimized_hybrid = VotingClassifier(
    estimators=[('random_forest', best_rf), ('xgboost', xgb.XGBClassifier(**best_xgb_params))],
    voting='soft',
    weights=[1, 2]  # Give more weight to XGBoost
)

# Train and evaluate
optimized_hybrid.fit(X_clean, y_clean)
y_pred = optimized_hybrid.predict(X_clean)

from sklearn.metrics import accuracy_score
accuracy_optimized = accuracy_score(y_clean, y_pred)
print(f"🚀 Optimized Hybrid Model Accuracy: {accuracy_optimized:.4f}")


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Predict on the test set
y_pred = hybrid_model.predict(X_test)

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

# Classification report (includes F1-score, precision, recall)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plot the confusion matrix
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Healthy', 'Tipburn'], yticklabels=['Healthy', 'Tipburn'])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score

# For ROC, we need predicted probabilities instead of class labels
# If your hybrid model is a VotingClassifier with 'soft' voting, use predict_proba
# Otherwise, we'll need to modify for individual models

# Check if hybrid model supports predict_proba
if hasattr(optimized_hybrid, "predict_proba"):
    y_probs = optimized_hybrid.predict_proba(X_test)[:, 1]  # Probability of positive class (Tipburn)
else:
    # If not, fallback to XGBoost probabilities as proxy
    y_probs = xgb_model.predict_proba(X_test)[:, 1]

# Compute ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_probs)

# Compute AUC
auc_score = roc_auc_score(y_test, y_probs)

# Plot ROC curve
plt.figure(figsize=(6,6))
plt.plot(fpr, tpr, color='blue', label=f'ROC Curve (AUC = {auc_score:.4f})')
plt.plot([0,1], [0,1], color='red', linestyle='--', label='Random Guess')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Hybrid Model')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()